# Md Abrar Jahin

# Modelling

In [1]:
#@title
import sklearn
from sklearn.model_selection import GridSearchCV
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances, silhouette_samples, silhouette_score, confusion_matrix, roc_curve, auc
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering 

from statsmodels.stats.outliers_influence import variance_inflation_factor
from time import time
from datetime import datetime
import matplotlib.cm as cm
from scipy.spatial.distance import cdist
from yellowbrick.cluster import KElbowVisualizer
from datetime import datetime
from joblib import Parallel, delayed

import seaborn as sns
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import joblib

#Import libraries:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from dask.distributed import Client

# It's recommended to use dask_cuda for GPU assignment
!pip install dask dask_cuda dask-ml 
!pip install tensorflow

from dask_cuda import LocalCUDACluster
from dask import array as da

%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2022.2.0
    Uninstalling dask-2022.2.0:
      Successfully uninstalled dask-2022.2.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2022.2.0
    Uninstalling distributed-2022.2.0:
      Successfully uninstalled distributed-2022.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling pro

In [2]:
#@title
import pandas as pd
import numpy as np
import gensim 
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('/kaggle/input/uk-twitter-covid19-dataset/sample_data.csv')

In [3]:
#@title
#after manual check
'''
negative_num=len(df[df['senti_vader'] < 0]): This line filters the DataFrame 'df' based on the condition df['senti_vader'] < 0, 
which selects all rows where the 'senti_vader' column has a value less than zero (negative). 
The len() function is then used to determine the number of rows in the filtered DataFrame, 
representing the count of negative values. The count is assigned to the variable 'negative_num'.
'''
negative_num=len(df[df['senti_vader'] < 0])
print("negative:", negative_num)
neutral_num=len(df[df['senti_vader'] == 0])
print("neutral", neutral_num)
positive_num=len(df[df['senti_vader'] > 0])
print("positive", positive_num) 

negative: 1000
neutral 1000
positive 1000


In [4]:
#@title
# Funtion for bag-of-words or BoW
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from nltk.tokenize import word_tokenize

def bag_of_words(df):
    bow_vectorizer = CountVectorizer() 
    #Transforming the input text data into a bag-of-words representation
    bow = bow_vectorizer.fit_transform(df['lemma_sentence(with POS)']) 
    #Printing the size of the vocabulary created by the vectorizer.
    print(len(bow_vectorizer.vocabulary_))
    #Returning the bag-of-words representation of the input data
    return bow

df_bow=bag_of_words(df)
df_bow.shape

7222


(3000, 7222)

In [5]:
#@title
# function for TF-IDF
def tf_idf(df):
    tf_idf_vectorizer = TfidfVectorizer(norm='l2') #extract features
    tfidf = tf_idf_vectorizer.fit_transform(df['lemma_sentence(with POS)']) #vectors
    return tfidf
df_tfidf=tf_idf(df)
df_tfidf.shape

(3000, 7222)

In [6]:
#@title
# function for Word2vec
#reference：https://www.pythonf.cn/read/93491
#https://github.com/Shwetago/Sentiment_Analysis/blob/master/Twitter_Sentiment_Analysis.ipynb
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
Tokenize_tweet = df['lemma_sentence(with POS)'].apply(word_tokenize) #Tokenizing the 'lemma_sentence(with POS)' column in the dataframe
print(Tokenize_tweet)

#Creating a Word2Vec model with specified parameters
Model_W2V = gensim.models.Word2Vec(Tokenize_tweet, vector_size=200, #features
                                   window=5, 
                                   min_count=1, 
                                   sg=1,  #skip-gram model
                                   hs=0,
                                   negative=10, 
                                   workers=2, 
                                   seed=34) 

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0       [kid, not, wana, homework, sparkle, covid, spa...
1       [not, mess, covid, wear, mask, time, london, u...
2       [problem, airfield, cost, remain, open, verse,...
3       [remain, astonished, stock, market, not, face,...
4       [dear, lord, jesus, christ, grace, listen, pra...
                              ...                        
2995    [lockdown, not, deal, covid, deal, stupidity, ...
2996    [regulator, refuse, approve, mass, covid, test...
2997    [covid, vaccine, week, ago, manchester, real, ...
2998    [coronavirustrain, covid, covid, wearamask, we...
2999    [news, cycle, number, covid, death, car, crash...
Name: lemma_sentence(with POS), Length: 3000, dtype: object


In [7]:
#@title
# Each word can get its own vector. The representation of a tweets can the vector sum of each word divided by the total number(average) 
# or just the sum of each word vector
def word2vec_tweet(tokens, size):
    vector=np.zeros(size).reshape((1,size)) #Initializing a zero-filled vector of given size
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size)) #Adding word vectors to the tweet vector
        vector_cnt += 1
    return vector/vector_cnt  #Calculating the average of tweet vectors

def word2vec_tweet_2(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size))
    return vector  #Returning the sum of tweet vectors

tweet_arr=np.zeros((len(Tokenize_tweet), 200)) #Initializing a zero-filled array for storing tweet vectors.

for i in range (len(Tokenize_tweet)):
    tweet_arr[i,:] = word2vec_tweet(Tokenize_tweet[i], 200) #Assigning the average tweet vector to the corresponding row in the array
tweet_vec_df = pd.DataFrame(tweet_arr)

In [8]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
import tensorflow_hub as hub
import tensorflow as tf

In [9]:
# parameters in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] #tree number
max_features = ['auto', 'sqrt','log2']
max_depth = [10,20,30,40,50]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5, 10]

# Create the param grid
param_grid_forest = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid_forest)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 5, 10]}


In [10]:
# parameters in Multinomial NB
param_grid_nb = {'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0],
                'fit_prior':[True, False]}
print(param_grid_nb)

{'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0], 'fit_prior': [True, False]}


In [11]:
#parameters in SVC
# c_list=list(range(1,51))
param_grid_svc = {'C': [1, 10, 100, 1000],
                  'kernel': ['linear','poly','rbf','sigmoid'],
                  'degree': [1,2,3,4]}
print(param_grid_svc)

{'C': [1, 10, 100, 1000], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4]}


In [12]:
#Parameters in XGBoost
learning_rate = [0.01, 0.1, 0.5, 1, 0.3]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [20,30,40,50]
min_child_weight = [1, 3, 5, 7]
gamma = [0.0, 0.1, 0.2, 0.3]
subsample = [0.6, 0.7, 0.8, 0.9]
colsample_bytree = [0.6, 0.7, 0.8, 0.9]
booster=['gbtree','gblinear']

#Create the param grid for XGBoost
param_grid_xgb = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_depth': max_depth,
'min_child_weight': min_child_weight,
'objective': ['multi:softmax','multi:softprob'],
'gamma': gamma,
'subsample': subsample,
'colsample_bytree': colsample_bytree,
'booster': booster}

print(param_grid_xgb)

{'learning_rate': [0.01, 0.1, 0.5, 1, 0.3], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_depth': [20, 30, 40, 50], 'min_child_weight': [1, 3, 5, 7], 'objective': ['multi:softmax', 'multi:softprob'], 'gamma': [0.0, 0.1, 0.2, 0.3], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'booster': ['gbtree', 'gblinear']}


In [13]:
#parameters in Gradient Boosting Machine (GBM)
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_features = ['auto', 'sqrt','log2']
max_depth = [10,20,30,40]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5, 10]

#Create the param grid for GBM
param_grid_gbm = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf}
print(param_grid_gbm)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 5, 10]}


In [14]:
#Parameters in LightGBM
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [5, 10, 15, 20]
num_leaves = [10, 20, 30, 40]
min_child_samples = [1, 3, 5, 7]
subsample = [0.6, 0.7, 0.8, 0.9]
colsample_bytree = [0.6, 0.7, 0.8, 0.9]

#Create the param grid for LightGBM
param_grid_lgbm = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_depth': max_depth,
'num_leaves': num_leaves,
'objective': ['multiclass'],
'min_child_samples': min_child_samples,
'subsample': subsample,
'colsample_bytree': colsample_bytree}
print(param_grid_lgbm)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_depth': [5, 10, 15, 20], 'num_leaves': [10, 20, 30, 40], 'objective': ['multiclass'], 'min_child_samples': [1, 3, 5, 7], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}


In [15]:
#parameters in CatBoost
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [10,20,30,40]
l2_leaf_reg = [1, 3, 5, 7, 9]

#Create the param grid for CatBoost
param_grid_catboost = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'loss_function': ['MultiClass'],
'max_depth': max_depth,
'l2_leaf_reg': l2_leaf_reg}
print(param_grid_catboost)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'loss_function': ['MultiClass'], 'max_depth': [10, 20, 30, 40], 'l2_leaf_reg': [1, 3, 5, 7, 9]}


In [16]:
#@title
# initialize the classifiers
from sklearn.metrics import classification_report 
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgbm
!pip install catboost
from catboost import CatBoostClassifier

model_forest = RandomForestClassifier()
model_nb = MultinomialNB()
model_svc = SVC()
model_xgb = xgb.XGBClassifier()
model_gbm = GradientBoostingClassifier()
model_lgbm = lgbm.LGBMClassifier()
model_cat = CatBoostClassifier()

# BoW/TFIDF/word2vec for classical models

In [17]:
#@title
#BoW for all classification models
#split the train and test datasets
from sklearn.model_selection import train_test_split
# df_bow, df_tfidf, tweet_vec_df
#x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2, random_state=42)
#x_train, x_test, y_train, y_test = train_test_split(df_tfidf, df['senti_textblob'],test_size = 0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(tweet_vec_df, df['senti_textblob'],test_size = 0.2, random_state=42)

In [18]:
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

# create an instance of the OneHotEncoder class
encoder = OneHotEncoder(sparse=False)

y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [26]:
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

from sklearn.preprocessing import MinMaxScaler

# Scale the input data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
        if(name!='xgb' and name!='nb'):
            RandomGrid.fit(x_train, y_train)
        elif(name=='nb'):
            RandomGrid.fit(x_train_scaled, y_train)
        else:
            RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
        print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))


Best params for rf:
{'n_estimators': 190, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 40}
Classification report for rf:
              precision    recall  f1-score   support

        -1.0       0.54      0.16      0.25       135
         0.0       0.61      0.67      0.64       238
         1.0       0.50      0.66      0.57       227

    accuracy                           0.55       600
   macro avg       0.55      0.50      0.49       600
weighted avg       0.55      0.55      0.53       600

Best params for nb:
{'fit_prior': True, 'alpha': 0.1}
Classification report for nb:
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       135
         0.0       0.44      0.02      0.03       238
         1.0       0.38      0.98      0.54       227

    accuracy                           0.38       600
   macro avg       0.27      0.33      0.19       600
weighted avg       0.32      0.38      0.22      

## RF ensembles

In [21]:
#@title
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using RandomForestClassifier, KNeighborsClassifier, and MLPClassifier 
as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV object to search over 
a parameter grid for the best hyperparameters. Finally, it prints the classification report for the predictions made on the test set.
'''

# define base models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('rf', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {'rf__max_depth': [20,30,40],
              'rf__max_features': ['sqrt'],
              'rf__min_samples_leaf': [1, 2, 5, 10],
              'rf__min_samples_split': [2, 5, 10, 15],
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'): # Enabling multiprocessing for parallel execution
    random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=5, mlp__alpha=0.001, mlp__learning_rate_init=0.001, rf__max_depth=40, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5; total time=  31.2s
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=5, mlp__alpha=0.001, mlp__learning_rate_init=0.001, rf__max_depth=40, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5; total time=  31.5s
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=0.001, mlp__learning_rate_init=0.1, rf__max_depth=40, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=15; total time=  17.0s
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=5, mlp__alpha=0.001, mlp__learning_rate_init=0.001, rf__max_d

## RF+bagging

In [ ]:
#@title
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
import xgboost as xgb
import sklearn
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)
def modelfit(x_train, y_train, x_test, y_test, params, early_stopping_rounds=100):        
    # Initialize the base classifier
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
    
    # Initialize the bagging classifier
    # Initializing a Bagging classifier with Random Forest as the base estimator and 10 estimators
    alg = BaggingClassifier(base_estimator=clf, n_estimators=10, random_state=0)
    
    # Performing randomized search cross-validation with Bagging classifier as the estimator and given parameter distributions.
    grid_xgb = sklearn.model_selection.RandomizedSearchCV(estimator=alg, param_distributions=params, cv=3, verbose=2, scoring='roc_auc')

    #Fit the algorithm on the data
    grid_xgb.fit(x_train, y_train)
    grid_xgb = grid_xgb
    grid_result = grid_xgb.best_estimator_
    
    #train_history = grid_result.evals_result()
    y_pred_test = grid_result.predict(x_test)
    y_predprob_test = grid_result.predict_proba(x_test)[:,-1] #Computing the probability of the positive class for each test sample

    return grid_xgb, grid_result, y_pred_test, y_predprob_test

from joblib import parallel_backend

param_grid = {
'n_estimators':[int(x) for x in np.linspace(start = 10, stop = 200, num = 20)],
}

#with LocalCUDACluster() as cluster:
#    with Client(cluster) as client:
with parallel_backend('multiprocessing'): #Enabling multiprocessing for parallel execution
        grid_xgb, grid_result_xgb, y_pred_test_xgb, y_predprob_test_xgb = modelfit(x_train, y_train, x_test, y_test, param_grid)

print(sklearn.metrics.classification_report(y_test, y_pred_test_xgb))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....................................n_estimators=40; total time=  55.0s
[CV] END ....................................n_estimators=40; total time=  55.6s
[CV] END ....................................n_estimators=40; total time=  55.0s
[CV] END ....................................n_estimators=80; total time= 1.8min
[CV] END ...................................n_estimators=200; total time= 4.6min
[CV] END ...................................n_estimators=200; total time= 4.6min
[CV] END ...................................n_estimators=200; total time= 4.6min


## RF+GB stacking and voting

In [24]:
#@title
'''
In this example, we are using a stacking classifier as the hybrid ensemble. We first define two base estimators, RandomForestClassifier and GradientBoostingClassifier, 
and then stack them using a StackingClassifier with a RandomForestClassifier as the final estimator. We also define a VotingClassifier that uses the same base estimators.
'''
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the base estimators
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Define the stacking estimator
estimators = [
    ('rf', rf),
    ('gb', gb)
]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier()) # stacking classifier combining rf and gb

# Define the voting classifier
voting_clf = VotingClassifier(estimators=estimators) # voting classifier using rf and gb

# Perform grid search to find the best hyperparameters for the stacking classifier
grid_search_stacking = RandomizedSearchCV(stacking_clf, param_distributions = param_grid_lgbm, cv=5, verbose=2)
with parallel_backend('multiprocessing'): # Enabling multiprocessing for parallel execution
    grid_search_stacking.fit(x_train, y_train)

    # Fit the voting classifier on the training data
    voting_clf.fit(x_train, y_train)

# Make predictions on the test data
y_pred_stacking = grid_search_stacking.predict(x_test)
y_pred_voting = voting_clf.predict(x_test)

# Print the accuracy scores
print('Stacking classifier CR: ', sklearn.metrics.classification_report(y_test, y_pred_stacking))
print('Voting classifier CR: ', sklearn.metrics.classification_report(y_test, y_pred_voting))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END final_estimator__max_depth=30, final_estimator__max_features=sqrt, final_estimator__min_samples_leaf=1, final_estimator__min_samples_split=5, final_estimator__n_estimators=80; total time=  51.3s
[CV] END final_estimator__max_depth=30, final_estimator__max_features=sqrt, final_estimator__min_samples_leaf=1, final_estimator__min_samples_split=5, final_estimator__n_estimators=80; total time=  51.6s
[CV] END final_estimator__max_depth=30, final_estimator__max_features=sqrt, final_estimator__min_samples_leaf=1, final_estimator__min_samples_split=5, final_estimator__n_estimators=80; total time=  49.9s
[CV] END final_estimator__max_depth=30, final_estimator__max_features=sqrt, final_estimator__min_samples_leaf=1, final_estimator__min_samples_split=5, final_estimator__n_estimators=80; total time=  50.0s
[CV] END final_estimator__max_depth=30, final_estimator__max_features=sqrt, final_estimator__min_samples_leaf=1, final_estima

In [ ]:
#@title
grid_search_stacking.best_params_

## LGBM ensemble

In [ ]:
#@title
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using LGBM Classifier, KNeighborsClassifier, and MLPClassifier as base models, and MLPClassifier as the final estimator. 
It then uses a GridSearchCV object to search over a parameter grid for the best hyperparameters. Finally, it prints the classification report for the predictions made on the test set.
'''
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# define base models
model1 = lgbm.LGBMClassifier(n_estimators=100, random_state=42, 
                          colsample_bytree = 0.6,
                          subsample = 0.8,
                          min_child_samples = 1,
                          objective = 'multiclass',
                          num_leaves = 20,
                          max_depth = 10,
                          learning_rate = 0.5
                         ) #RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('lg', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
      random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=7, mlp__alpha=0.1, mlp__learning_rate_init=0.01; total time= 1.6min
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=7, mlp__alpha=0.1, mlp__learning_rate_init=0.01; total time= 1.6min
[CV] END final_estimator__alpha=0.01, final_estimator__learning_rate_init=0.01, knn__n_neighbors=3, mlp__alpha=0.1, mlp__learning_rate_init=0.001; total time= 3.9min
[CV] END final_estimator__alpha=0.01, final_estimator__learning_rate_init=0.01, knn__n_neighbors=3, mlp__alpha=0.1, mlp__learning_rate_init=0.001; total time= 3.9min
[CV] END final_estimator__alpha=0.01, final_estimator__learning_rate_init=0.01, knn__n_neighbors=3, mlp__alpha=0.1, mlp__learning_rate_init=0.001; total time= 4.2min
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=7, mlp__alpha=0.1, mlp_

# BERT

In [24]:
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #Checking for CUDA availability to decide the device

#Loading a pre-trained DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

#Tokenizing the input sentences, padding and truncating them
df_bert = tokenizer(df['lemma_sentence(with POS)'].values.tolist(), padding = True, truncation = True, return_tensors="pt")

print(df_bert.keys())

#Moving the tokenized input to the available device (GPU)
tokenized_train = {k:torch.tensor(v).to(device) for k,v in df_bert.items()}

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


dict_keys(['input_ids', 'attention_mask'])


In [25]:
with torch.no_grad(): #Context manager to disable gradient calculation for efficiency
    #"model(**tokenized_train)": Model inference using tokenized input
    #"hidden_train = ...": Obtaining hidden states from the model's output
    hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]

#get only the [CLS] hidden states
cls_train = hidden_train.last_hidden_state[:,0,:] #Extracting only the [CLS] token's hidden state for each sentence

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cls_train.to("cpu"), df['senti_textblob'],test_size = 0.2)
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

torch.Size([2400, 768]) (2400,) torch.Size([600, 768]) (600,)


In [27]:
x_train = np.array(x_train, dtype='float32')
y_train = np.array(y_train, dtype='float32')
x_test = np.array(x_test, dtype='float32')
y_test = np.array(y_test, dtype='float32')

In [ ]:
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

from sklearn.preprocessing import MinMaxScaler

# Scale the input data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
        if(name!='xgb' and name!='nb'):
            RandomGrid.fit(x_train, y_train)
        elif(name=='nb'):
            RandomGrid.fit(x_train_scaled, y_train)
        else:
            RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
        print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))


Best params for rf:

{'n_estimators': 190, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 40}

Classification report for rf:

              precision    recall  f1-score   support



        -1.0       0.40      0.14      0.20       125

         0.0       0.67      0.70      0.68       257

         1.0       0.53      0.70      0.60       218



    accuracy                           0.58       600

   macro avg       0.53      0.51      0.49       600

weighted avg       0.56      0.58      0.55       600



Best params for nb:

{'fit_prior': True, 'alpha': 0.5}

Classification report for nb:

              precision    recall  f1-score   support



        -1.0       0.38      0.44      0.41       125

         0.0       0.62      0.63      0.62       257

         1.0       0.51      0.45      0.48       218



    accuracy                           0.53       600

   macro avg       0.50      0.51      0.50       600

weighted avg       0.53  

In [28]:
#RF+KNN+MLP
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using RandomForestClassifier, KNeighborsClassifier, and MLPClassifier 
as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV object to 
search over a parameter grid for the best hyperparameters. Finally, it prints the classification report for the predictions made on the test set.
'''

# define base models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('rf', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {'rf__max_depth': [20,40,60],
              'rf__max_features': ['sqrt'],
              'rf__min_samples_leaf': [1, 2, 5, 10],
              'rf__min_samples_split': [2, 5, 10, 15],
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
    random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END final_estimator__alpha=0.01, final_estimator__learning_rate_init=0.001, knn__n_neighbors=3, mlp__alpha=1, mlp__learning_rate_init=0.01, rf__max_depth=60, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10; total time=  37.6s
[CV] END final_estimator__alpha=0.01

In [29]:
#@title
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using LGBM Classifier, KNeighborsClassifier, and MLPClassifier 
as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV object to search 
over a parameter grid for the best hyperparameters. Finally, it prints the classification report for the predictions made on the test set.
'''
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# define base models
model1 = lgbm.LGBMClassifier(n_estimators=100, random_state=42, 
                          colsample_bytree = 0.6,
                          subsample = 0.8,
                          min_child_samples = 1,
                          objective = 'multiclass',
                          num_leaves = 20,
                          max_depth = 10,
                          learning_rate = 0.5
                         ) #RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('lg', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
      random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[CV] END final_estimator__alpha=0.01, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=0

# roBERTa

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = AutoModel.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest').to(device)

df_bert = tokenizer(df['lemma_sentence(with POS)'].values.tolist(), padding = True, truncation = True, return_tensors="pt")

print(df_bert.keys())

#move on device (GPU)
tokenized_train = {k:torch.tensor(v).to(device) for k,v in df_bert.items()}

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


dict_keys(['input_ids', 'attention_mask'])


In [18]:
with torch.no_grad():
    hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]

#get only the [CLS] hidden states
cls_train = hidden_train.last_hidden_state[:,0,:]

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cls_train.to("cpu"), df['senti_textblob'],test_size = 0.2)
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

torch.Size([2400, 768]) (2400,) torch.Size([600, 768]) (600,)


In [20]:
x_train = np.array(x_train, dtype='float32')
y_train = np.array(y_train, dtype='float32')
x_test = np.array(x_test, dtype='float32')
y_test = np.array(y_test, dtype='float32')

In [ ]:
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

from sklearn.preprocessing import MinMaxScaler

# Scale the input data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
        if(name!='xgb' and name!='nb'):
            RandomGrid.fit(x_train, y_train)
        elif(name=='nb'):
            RandomGrid.fit(x_train_scaled, y_train)
        else:
            RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
        print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
#RF+KNN+MLP
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using RandomForestClassifier, KNeighborsClassifier, and MLPClassifier 
as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV object 
to search over a parameter grid for the best hyperparameters. Finally, it prints the classification 
report for the predictions made on the test set.
'''

# define base models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('rf', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {'rf__max_depth': [20,40,60],
              'rf__max_features': ['sqrt'],
              'rf__min_samples_leaf': [1, 2, 5, 10],
              'rf__min_samples_split': [2, 5, 10, 15],
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
    random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


In [ ]:
#@title
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using LGBM, KNeighborsClassifier, and MLPClassifier as base models, 
and MLPClassifier as the final estimator. It then uses a GridSearchCV object to search over a parameter grid for 
the best hyperparameters. Finally, it prints the classification report for the predictions made on the test set.
'''
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# define base models
model1 = lgbm.LGBMClassifier(n_estimators=100, random_state=42, 
                          colsample_bytree = 0.6,
                          subsample = 0.8,
                          min_child_samples = 1,
                          objective = 'multiclass',
                          num_leaves = 20,
                          max_depth = 10,
                          learning_rate = 0.5
                         ) #RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('lg', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
      random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


# SBERT

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(device)

df_bert = tokenizer(df['lemma_sentence(with POS)'].values.tolist(), padding = True, truncation = True, return_tensors="pt")

print(df_bert.keys())

#move on device (GPU)
tokenized_train = {k:torch.tensor(v).to(device) for k,v in df_bert.items()}

In [ ]:
with torch.no_grad():
    hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]

#get only the [CLS] hidden states
cls_train = hidden_train.last_hidden_state[:,0,:]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cls_train.to("cpu"), df['senti_textblob'],test_size = 0.2)
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
x_train = np.array(x_train, dtype='float32')
y_train = np.array(y_train, dtype='float32')
x_test = np.array(x_test, dtype='float32')
y_test = np.array(y_test, dtype='float32')

In [23]:
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

from sklearn.preprocessing import MinMaxScaler

# Scale the input data
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
        if(name!='xgb' and name!='nb'):
            RandomGrid.fit(x_train, y_train)
        elif(name=='nb'):
            RandomGrid.fit(x_train_scaled, y_train)
        else:
            RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
        print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [24]:
#RF+KNN+MLP
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using RandomForestClassifier, KNeighborsClassifier, and 
MLPClassifier as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV 
object to search over a parameter grid for the best hyperparameters. 
Finally, it prints the classification report for the predictions made on the test set.
'''

# define base models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('rf', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {'rf__max_depth': [20,40,60],
              'rf__max_features': ['sqrt'],
              'rf__min_samples_leaf': [1, 2, 5, 10],
              'rf__min_samples_split': [2, 5, 10, 15],
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
    random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=0.001, mlp__learning_rate_init=0.1, rf__max_depth=60, rf__max_features=sqrt, rf__min_samples_leaf=10, rf__min_samples_split=5; total time=  28.1s
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=0.001, mlp__learning_rate_init=0.1, rf__max_depth=60, rf__max_features=sqrt, rf__min_samples_leaf=10, rf__min_samples_split=5; total time=  30.4s
[CV] END final_estimator__alpha=0.001, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=0.001, mlp__learning_rate_init=0.1, rf__max_depth=60, rf__max_features=sqrt, rf__min_samples_leaf=10, rf__min_samples_split=5; total time=  30.3s
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.001, knn__n_neighbors=5, mlp__alpha=0.001, mlp__learning_rate_init=0.1, rf__max_depth=20,

In [25]:
#@title
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend

'''
This code defines a stacking ensemble using LGBM, KNeighborsClassifier, and MLPClassifier 
as base models, and MLPClassifier as the final estimator. It then uses a GridSearchCV object 
to search over a parameter grid for the best hyperparameters. Finally, it prints 
the classification report for the predictions made on the test set.
'''
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# define base models
model1 = lgbm.LGBMClassifier(n_estimators=100, random_state=42, 
                          colsample_bytree = 0.6,
                          subsample = 0.8,
                          min_child_samples = 1,
                          objective = 'multiclass',
                          num_leaves = 20,
                          max_depth = 10,
                          learning_rate = 0.5
                         ) #RandomForestClassifier(n_estimators=100, random_state=42)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = MLPClassifier(hidden_layer_sizes=(32, 32), random_state=42)

# define stacking ensemble
estimators = [('lg', model1),
              ('knn', model2),
              ('mlp', model3)]

clf = StackingClassifier(estimators=estimators, 
                         final_estimator=MLPClassifier(hidden_layer_sizes=(64, 64),
                                                       random_state=42))


# define parameter grid for randomized search
param_dist = {
              'knn__n_neighbors': [3, 5, 7],
              'mlp__alpha': [0.001, 0.01, 0.1, 1],
              'mlp__learning_rate_init': [0.001, 0.01, 0.1],
              'final_estimator__alpha': [0.001, 0.01, 0.1, 1],
              'final_estimator__learning_rate_init': [0.001, 0.01, 0.1]}

# define RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=clf, 
                             param_distributions=param_dist, 
                             cv=3, 
                             n_jobs=-1, 
                             verbose=2)

# fit the RandomizedSearchCV object to the data
with parallel_backend('multiprocessing'):
      random_search.fit(x_train, y_train)

# make predictions on test set
y_pred = random_search.predict(x_test)

# print classification report
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=5, mlp__alpha=0.01, mlp__learning_rate_init=0.1; total time=  43.9s
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=5, mlp__alpha=0.01, mlp__learning_rate_init=0.1; total time=  40.0s
[CV] END final_estimator__alpha=1, final_estimator__learning_rate_init=0.01, knn__n_neighbors=5, mlp__alpha=0.01, mlp__learning_rate_init=0.1; total time=  43.0s
[CV] END final_estimator__alpha=0.1, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=1, mlp__learning_rate_init=0.001; total time= 1.3min
[CV] END final_estimator__alpha=0.1, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=1, mlp__learning_rate_init=0.001; total time= 1.3min
[CV] END final_estimator__alpha=0.1, final_estimator__learning_rate_init=0.001, knn__n_neighbors=7, mlp__alpha=1, mlp__learni